In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime
from functools import partial

todo_items = []

# -------------------- 공통 레이아웃 --------------------

layout_wide = widgets.Layout(width='750px')
layout_large = widgets.Layout(width='180px')
layout_medium = widgets.Layout(width='150px')
layout_small = widgets.Layout(width='100px')

# 진행률 바 + 라벨
overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, bar_style='info', layout=layout_medium)
overall_progress_label = widgets.Label(value='(0%)', layout=layout_small)

# 필터 및 설정
tag_filter_dropdown = widgets.Dropdown(options=["전체"], value="전체", description="태그:", layout=layout_medium)
sort_dropdown = widgets.Dropdown(options=["마감일", "진행률", "중요도"], value="마감일", description="정렬:", layout=layout_medium)
hide_completed_toggle = widgets.ToggleButton(value=False, description='완료 항목 숨기기', icon='eye-slash', layout=layout_medium)

todo_list_box = widgets.VBox(layout=widgets.Layout(border='1px solid #ccc', padding='10px', width='100%'))

# -------------------- 유틸 함수 --------------------

def update_overall_progress():
    visible_items = [i for i in todo_items if not (hide_completed_toggle.value and i['progress'] == 100)]
    if not visible_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in visible_items) / len(visible_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"

def refresh_list(*args):
    todo_list_box.children = []

    # 헤더 추가
    headers = ['할 일', '진행률', '입력', '마감일', '해시태그', '중요도', '완료', '삭제']
    header_layouts = [layout_large, layout_medium, layout_medium, layout_medium, layout_medium, layout_medium, layout_medium, layout_medium]
    header_row = widgets.HBox([widgets.Label(h, layout=l) for h, l in zip(headers, header_layouts)])
    rows = [header_row]

    filtered = [i for i in todo_items if tag_filter_dropdown.value in ['전체', i['tag']]]
    if hide_completed_toggle.value:
        filtered = [i for i in filtered if i['progress'] < 100]

    if sort_dropdown.value == '마감일':
        filtered.sort(key=lambda x: x['due'] or datetime.max)
    elif sort_dropdown.value == '진행률':
        filtered.sort(key=lambda x: x['progress'])
    elif sort_dropdown.value == '중요도':
        priority_order = {'높음': 0, '보통': 1, '낮음': 2}
        filtered.sort(key=lambda x: priority_order.get(x['priority'], 1))

    for item in filtered:
        title = widgets.Text(value=item['title'], layout=layout_large, disabled=item['progress']==100)
        progress_bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=layout_medium)
        progress_input = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_medium)
        due = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-', layout=layout_medium)
        tag = widgets.Label(value=f"#{item['tag']}" if item['tag'] else '', layout=layout_medium)
        priority = widgets.Label(value=item['priority'], layout=layout_medium)
        complete_btn = widgets.Button(description='완료', icon='check', button_style='success', layout=layout_medium)
        delete_btn = widgets.Button(description='삭제', icon='times', button_style='danger', layout=layout_medium)

        def on_progress_change(change, item=item, bar=progress_bar):
            item['progress'] = change['new']
            bar.value = change['new']
            update_overall_progress()
            refresh_list()

        def on_delete_clicked(b, item=item):
            todo_items.remove(item)
            update_overall_progress()
            refresh_list()

        def on_complete_clicked(b, item=item):
            item['progress'] = 100
            update_overall_progress()
            refresh_list()

        progress_input.observe(partial(on_progress_change, item=item), names='value')
        delete_btn.on_click(partial(on_delete_clicked, item=item))
        complete_btn.on_click(partial(on_complete_clicked, item=item))

        row = widgets.HBox([title, progress_bar, progress_input, due, tag, priority, complete_btn, delete_btn], layout=widgets.Layout(margin='5px 0'))
        if item['progress'] == 100:
            row.layout.border = '1px solid #aaa'
            title.style = {'text_color': '#666'}
        rows.append(row)
    todo_list_box.children = rows

# -------------------- 입력 폼 --------------------

title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=layout_large)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_medium)
due_input = widgets.DatePicker(layout=layout_medium)
tag_input = widgets.Text(placeholder='#해시태그 입력', layout=layout_medium, description='해시태그:')
priority_input = widgets.Dropdown(options=["낮음", "보통", "높음"], value="보통", layout=layout_medium)
add_button = widgets.Button(description='추가', button_style='success', layout=layout_medium)

def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'tag': tag_input.value.strip(),
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        tag_input.value = ''
        priority_input.value = '보통'
        update_overall_progress()
        tags = sorted(set(i['tag'] for i in todo_items if i['tag']))
        tag_filter_dropdown.options = ['전체'] + tags
        refresh_list()

add_button.on_click(add_task)
tag_filter_dropdown.observe(refresh_list, names='value')
sort_dropdown.observe(refresh_list, names='value')
hide_completed_toggle.observe(refresh_list, names='value')

# -------------------- 저장/불러오기 --------------------

def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([
            {
                **i,
                'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None
            } for i in todo_items
        ], f)

def load_data(b):
    try:
        with open('todo_data.json', 'r') as f:
            raw = json.load(f)
            todo_items.clear()
            for i in raw:
                todo_items.append({
                    'title': i['title'],
                    'progress': i['progress'],
                    'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                    'tag': i['tag'],
                    'priority': i.get('priority', '보통')
                })
            update_overall_progress()
            refresh_list()
    except:
        print("불러오기 실패")

save_btn = widgets.Button(description='저장', button_style='info', layout=layout_medium)
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=layout_medium)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

# -------------------- UI 출력 --------------------

display(widgets.VBox([
    widgets.HBox([title_input, progress_input, due_input]),
    widgets.HBox([tag_input, priority_input, add_button]),
    widgets.HBox([save_btn, load_btn]),
    widgets.HBox([tag_filter_dropdown, sort_dropdown, hide_completed_toggle]),
    widgets.HBox([overall_progress_bar, overall_progress_label]),
    todo_list_box
]))

update_overall_progress()
refresh_list()
